In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import parse_raw_to_csv
import os
import numpy as np

df = pd.DataFrame()

N = [1,2,5,8,13]
n = [1,4,9,16,25]

for ii, i in enumerate(N):
    df_tmp = pd.DataFrame()
    for f in os.listdir("../../experiments/%d_fpga" % i):
        if any(f.endswith(e) for e in [".txt",".out"]) and any(f.startswith(s) for s in ["ptrans"]):
            df_parsed = parse_raw_to_csv.parse_file_or_folder(os.path.join("../../experiments/%d_fpga" % i, f), parse_raw_to_csv.parse_map.values())
            if "pcie" in f:
                df_parsed["type"] = "PCIE"
                df_parsed["norm_flops"] = float(df_parsed["best_mem_bw"]) / ((n[ii]))
                df_parsed["norm_bw"] = 32768 ** 2 * 4 * 3 / float(df_parsed["avg_calc_time"]) / ((n[ii]))
            else:
                df_parsed["type"] = "IEC"
                df_parsed["norm_flops"] = float(df_parsed["best_mem_bw"]) / ((n[ii]))
                df_parsed["norm_bw"] = 32768 ** 2 * 4 * 3 / float(df_parsed["avg_calc_time"]) / ((n[ii]))
            df_tmp = df_tmp.append(df_parsed)
    df_tmp["block_size"] = 512
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df_tmp["size"] = (df_tmp["size"].astype(int) / np.sqrt(n[ii])).astype(int)
    df = df.append(df_tmp)

for ii, i in enumerate([1,2]):
    df_tmp = pd.DataFrame()
    for f in os.listdir("../../experiments/%d_fpga/u280" % i):
        if any(f.endswith(e) for e in [".txt",".out"]) and any(f.startswith(s) for s in ["ptrans"]):
            df_parsed = parse_raw_to_csv.parse_file_or_folder(os.path.join("../../experiments/%d_fpga/u280" % i, f), parse_raw_to_csv.parse_map.values())
            df_parsed["type"] = "PCIE"
            df_parsed["norm_flops"] = float(df_parsed["best_mem_bw"]) / ((n[ii]))
            df_parsed["norm_bw"] = 32768 ** 2 * 4 * 3 / float(df_parsed["avg_calc_time"]) / ((n[ii]))
            df_tmp = df_tmp.append(df_parsed)
    df_tmp["block_size"] = 256
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df_tmp["size"] = (df_tmp["size"].astype(int) / np.sqrt(n[ii])).astype(int)
    df = df.append(df_tmp)


df["benchmark"] = df.index

df["size"] = (df["size"] / (df["block_size"] * np.sqrt(df["n"])).astype(int)).astype(int) * (df["block_size"] * np.sqrt(df["n"])).astype(int)
df["flops"] = df["size"].astype(int) ** 2 / df["best_calc_time"].astype(float)
df = df.reset_index()
df = df.apply(pd.to_numeric, errors='ignore')

df

In [ ]:
df_strongt = df
df_strong = df_strongt[df_strongt["size"] <= 34816]
df_weak = df_strongt[(df_strongt["size"] > 34816) | (df_strongt["n"] == 1)]

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, ptrans = plt.subplots(1)
ptrans.scatter(df_weak[df_weak["type"] == "PCIE"]["n"], df_weak[df_weak["type"] == "PCIE"]["best_mem_bw"] * 1.0e-9 / df_weak[df_weak["type"] == "PCIE"]["n"], label="Measurements d=168", marker="x")
#ptrans.scatter(df_strong["n"], df_strong["norm_flops"] * 1.0e-9, label="Measurements d=64", marker="x")
#ptrans.scatter(df_weak["n"], df_weak["best_calc_flops"] * 1.0e-9, label="weak")
#hpl.plot(np.linspace(0,5), [perf_last(48,n) * 1.0e-3 for n in np.linspace(0,5)], label="Model")
single_perf = df_strong[df_strong["N"] == 2]["norm_flops"] * 1.0e-9
#ptrans.plot(np.linspace(0,26), [single_perf * (n - 1)/2 for n in np.linspace(0,26)], label="Scaling Model")
#ptrans.annotate("(n - 1)/2 ⋅ %3.1f GFLOP/s" % (single_perf), (4,30))
ptrans.set_xlabel("Number FPGAs (n)")
ptrans.set_ylabel("GB/s")
#ptrans.set_xlim((0,26))
#ptrans.set_ylim((0,60))
ptrans.legend()
fig.savefig("ptrans_scaling_norm.pdf")

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, ptrans = plt.subplots(nrows=1,ncols=2)

n=1

ptrans_scaling_strong = ptrans[0]

# ptrans_scaling.scatter(df_weak[(df_weak["type"] == "PCIE") & (~df_weak["device"].str.contains("xilinx"))]["n"], df_weak[(df_weak["type"] == "PCIE") & (~df_weak["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (~df_weak["device"].str.contains("xilinx"))]["flops"].max() / n), marker="x", color="C0", label="520N PCIE weak")
# ptrans_scaling.scatter(df_weak[df_weak["type"] == "IEC"]["n"], df_weak[df_weak["type"] == "IEC"]["flops"] / (df_weak[(df_weak["type"] == "IEC") & (df_weak["n"] == n)]["flops"].max() / n), marker="x", color="C1", label="520N IEC weak")
# ptrans_scaling.scatter(df_weak[(df_weak["type"] == "PCIE") & (df_weak["device"].str.contains("xilinx"))]["n"], df_weak[(df_weak["type"] == "PCIE") & (df_weak["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (df_weak["device"].str.contains("xilinx"))]["flops"].max() / n), marker="x", color="C2", label="U280 PCIE weak")
ptrans_scaling_strong.scatter(df_strong[(df_strong["type"] == "PCIE") & (~df_strong["device"].str.contains("xilinx"))]["n"], df_strong[(df_strong["type"] == "PCIE") & (~df_strong["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (~df_weak["device"].str.contains("xilinx"))]["flops"].max() / n),marker="x", color="C0", label="Nallatech 520N")
ptrans_scaling_strong.scatter(df_strong[df_strong["type"] == "IEC"]["n"], df_strong[df_strong["type"] == "IEC"]["flops"] / (df_strong[(df_strong["type"] == "IEC") & (df_strong["n"] == n)]["flops"].max() / n),marker="x", color="C1",  label="Nallatech 520N IEC")
ptrans_scaling_strong.scatter(df_strong[(df_strong["type"] == "PCIE") & (df_strong["device"].str.contains("xilinx"))]["n"], df_strong[(df_strong["type"] == "PCIE") & (df_strong["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (df_weak["device"].str.contains("xilinx"))]["flops"].max() / n),marker="x", color="C2", label="Xilinx Alveo U280")
ptrans_scaling_strong.set_title("Strong Scaling")
ptrans_scaling_strong.set_xlabel("Number FPGAs (n)")
ptrans_scaling_strong.set_ylabel("Speedup")
ptrans_scaling_strong.plot(np.linspace(0,26), np.linspace(0,26), linestyle=":", color="black", label="optimal scaling")

ptrans_scaling_weak = ptrans[1]

ptrans_scaling_weak.scatter(df_weak[(df_weak["type"] == "PCIE") & (~df_weak["device"].str.contains("xilinx"))]["n"], df_weak[(df_weak["type"] == "PCIE") & (~df_weak["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (~df_weak["device"].str.contains("xilinx"))]["flops"].max() / n), marker="x", color="C0")
ptrans_scaling_weak.scatter(df_weak[df_weak["type"] == "IEC"]["n"], df_weak[df_weak["type"] == "IEC"]["flops"] / (df_weak[(df_weak["type"] == "IEC") & (df_weak["n"] == n)]["flops"].max() / n), marker="x", color="C1")
ptrans_scaling_weak.scatter(df_weak[(df_weak["type"] == "PCIE") & (df_weak["device"].str.contains("xilinx"))]["n"], df_weak[(df_weak["type"] == "PCIE") & (df_weak["device"].str.contains("xilinx"))]["flops"] / (df_weak[(df_weak["type"] == "PCIE") & (df_weak["n"] == n) & (df_weak["device"].str.contains("xilinx"))]["flops"].max() / n), marker="x", color="C2")
ptrans_scaling_weak.set_title("Weak Scaling")
ptrans_scaling_weak.set_xlabel("Number FPGAs (n)")
ptrans_scaling_weak.set_ylabel("Speedup")
ptrans_scaling_weak.plot(np.linspace(0,26), np.linspace(0,26), linestyle=":", color="black")

#ptrans.set_xlim((0,26))
#ptrans.set_ylim((0,60))
lgd = fig.legend(loc="center left", bbox_to_anchor=(0.97,0.5))
fig.tight_layout()
fig.savefig("ptrans_scaling_norm.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
df["best_calc_time"]